In [2]:
import pandas as pd
import numpy as np

In [3]:
customers = pd.read_csv('/content/drive/MyDrive/1_Python_practices/final_project_data/cleaned_customers.csv')
sellers = pd.read_csv('/content/drive/MyDrive/1_Python_practices/final_project_data/cleaned_sellers.csv')
orders = pd.read_csv('/content/drive/MyDrive/1_Python_practices/final_project_data/olist_orders_dataset.csv')
order_items = pd.read_csv('/content/drive/MyDrive/1_Python_practices/final_project_data/olist_order_items_dataset.csv')
cleaned_geolocation_customers = pd.read_csv('/content/drive/MyDrive/1_Python_practices/final_project_data/cleaned_geolocation.csv')
cleaned_geolocation_sellers = pd.read_csv('/content/drive/MyDrive/1_Python_practices/final_project_data/cleaned_geolocation.csv')

In [4]:
customers = customers.merge(cleaned_geolocation_customers, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
orders = orders.merge(customers, on='customer_id', how='left')
order_items = order_items.merge(orders, on='order_id', how='left')
sellers = sellers.merge(cleaned_geolocation_sellers, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
order_items = order_items.merge(sellers, on='seller_id', how='left')

In [5]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [7]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 31 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       112650 non-null  object 
 1   order_item_id                  112650 non-null  int64  
 2   product_id                     112650 non-null  object 
 3   seller_id                      112650 non-null  object 
 4   shipping_limit_date            112650 non-null  object 
 5   price                          112650 non-null  float64
 6   freight_value                  112650 non-null  float64
 7   customer_id                    112650 non-null  object 
 8   order_status                   112650 non-null  object 
 9   order_purchase_timestamp       112650 non-null  object 
 10  order_approved_at              112635 non-null  object 
 11  order_delivered_carrier_date   111456 non-null  object 
 12  order_delivered_customer_date 

In [9]:
order_items['distance_km'] = haversine(
    order_items['geolocation_lat_x'],
    order_items['geolocation_lng_x'],
    order_items['geolocation_lat_y'],
    order_items['geolocation_lng_y']
)

orders_distance = order_items.groupby('order_id').first().reset_index()

orders = orders.merge(orders_distance[['order_id', 'distance_km']], on='order_id', how='left')

orders.to_csv('orders_with_distances.csv', index=False)


In [10]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       99441 non-null  object 
 1   customer_id                    99441 non-null  object 
 2   order_status                   99441 non-null  object 
 3   order_purchase_timestamp       99441 non-null  object 
 4   order_approved_at              99281 non-null  object 
 5   order_delivered_carrier_date   97658 non-null  object 
 6   order_delivered_customer_date  96476 non-null  object 
 7   order_estimated_delivery_date  99441 non-null  object 
 8   customer_unique_id             99441 non-null  object 
 9   customer_zip_code_prefix       99441 non-null  int64  
 10  customer_city                  99441 non-null  object 
 11  customer_state                 99441 non-null  object 
 12  geolocation_zip_code_prefix    99441 non-null 

In [11]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,distance_km_x,distance_km_y
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,3149,-23.577482,-46.587077,são paulo,SP,18.538322,18.538322
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,47813,-12.186877,-44.540232,barreiras,BA,849.520470,849.520470
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,75265,-16.745150,-48.514783,vianopolis,GO,514.407596,514.407596
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,59296,-5.774002,-35.270976,sao goncalo do amarante,RN,1822.132331,1822.132331
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,9195,-23.676257,-46.514580,santo andre,SP,29.765008,29.765008


In [13]:
orders = orders[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
                 'order_approved_at', 'order_delivered_carrier_date',
                 'order_delivered_customer_date', 'order_estimated_delivery_date', 'distance_km_x']]


In [14]:
orders.to_csv('orders_with_distances.csv', index=False)

In [15]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       99441 non-null  object 
 1   customer_id                    99441 non-null  object 
 2   order_status                   99441 non-null  object 
 3   order_purchase_timestamp       99441 non-null  object 
 4   order_approved_at              99281 non-null  object 
 5   order_delivered_carrier_date   97658 non-null  object 
 6   order_delivered_customer_date  96476 non-null  object 
 7   order_estimated_delivery_date  99441 non-null  object 
 8   distance_km_x                  98666 non-null  float64
dtypes: float64(1), object(8)
memory usage: 6.8+ MB
